In [1]:
import pandas as pd
import json
from PIL import Image
import numpy as np


In [2]:

# export 2
df1 = pd.read_csv('train_data/raw/vott-csv-export-2/train-export.csv')
df1 = df1[df1['label'].str.contains('plate')]
height = []
width = []
for i in range(len(df1)):
    img = Image.open('train_data/raw/vott-csv-export-2/' + df1.iloc[i]['image'])
    height.append(img.height)
    width.append(img.width)
    res = max(height[i], width[i])
    img.save('train_data/raw/training_images/' + df1.iloc[i]['image'].split('.')[0] + '.png')
    
df1['height'] = height
df1['width'] = width

df1[['xmin', 'xmax']] = df1[['xmin', 'xmax']].apply(lambda x: x/df1['width'])
df1[['ymin', 'ymax']] = df1[['ymin', 'ymax']].apply(lambda x: x/df1['height'])
df1['image'] = df1['image'].apply(lambda x: x.split('.')[0] + '.png')

df1.drop(['width', 'height'], axis=1, inplace=True)

In [3]:
df1

,image,xmin,ymin,xmax,ymax,label
0,26l6uc3en3621.png,0.531514,0.285266,0.888476,0.660658,outside_plate
1,26l6uc3en3621.png,0.100290,0.131661,0.349235,0.373041,inside_plate
3,8c823bb58d8cb48e2d6cad6c69ed758d.png,0.606823,0.374608,1.000000,0.703762,outside_plate
4,8c823bb58d8cb48e2d6cad6c69ed758d.png,0.000000,0.368339,0.306717,0.677116,inside_plate
7,a5f12582ddc4f11e49f0443396f2b09b.png,0.106191,0.547806,0.317790,0.847962,outside_plate
...,...,...,...,...,...,...
158,teen1.png,0.609457,0.178683,0.706585,0.394201,inside_plate
162,Woman-squats-40kg-whilst-26-weeks-pregnant.png,0.149573,0.116771,0.411026,0.380094,inside_plate
163,Woman-squats-40kg-whilst-26-weeks-pregnant.png,0.536619,0.247649,0.870742,0.613636,outside_plate
165,wiod70d8ejrinysdv22l.png,0.238164,0.577456,0.379244,0.859590,outside_plate


In [4]:
# export 1
df2 = pd.read_csv('train_data/raw/vott-csv-export/bar-tracking-export.csv')
df2.iloc[:, 1:5] = df2.iloc[:, 1:5]/720
df = pd.concat((df1, df2))

df['height'] = df['ymax'] - df['ymin']
df['width'] = df['xmax'] - df['xmin']

annotations = {}
for i in range(len(df)):
    fn = df.iloc[i]['image'].split('.')[0]
    side = df.iloc[i]['label']
    
    if fn not in annotations:
        annotations[fn] = {}
        
    annotations[fn][side] = {
        'box': [df.iloc[i]['xmin'], df.iloc[i]['ymin'], df.iloc[i]['xmax'], df.iloc[i]['ymax']],
        'center': [df.iloc[i]['xmin'] + df.iloc[i]['width']/2, df.iloc[i]['ymin'] + df.iloc[i]['height']/2],
        'height': df.iloc[i]['height'],
        'width': df.iloc[i]['width']
    }
    
    annotations[fn][side]['partial_height'] = 1 if annotations[fn][side]['box'][1] <= 0.00001 or annotations[fn][side]['box'][3] >= 0.99999 else 0
    annotations[fn][side]['partial_width'] = 1 if annotations[fn][side]['box'][0] <= 0.00001 or annotations[fn][side]['box'][2] >= 0.99999 else 0

print('Starting with', len(annotations), 'annotations')
    
keys = list(annotations.keys())
for id in keys:
    remove = False
    if len(annotations[id]) != 2:
        print('Missing annotation for ', id, ', removing entry...', sep='')
        remove = True
    else:
        if annotations[id]['inside_plate']['partial_width'] and annotations[id]['outside_plate']['partial_width']:
            print('Partial width on both sides for ', id, ', removing entry...', sep='')
            remove = True
            
        if annotations[id]['inside_plate']['partial_height'] and annotations[id]['outside_plate']['partial_height']:
            print('Partial height on both sides for ', id, ', removing entry...', sep='')
            remove = True
        
    if remove:
        del annotations[id]
        
print('Ending with', len(annotations), 'annotations')

keys = list(annotations.keys())
for id in keys:
    if annotations[id]['inside_plate']['box'][0] <= 0.00001:
        annotations[id]['inside_plate']['box'][0] = annotations[id]['inside_plate']['box'][2] - annotations[id]['outside_plate']['width']
    if annotations[id]['inside_plate']['box'][1] <= 0.00001:
        annotations[id]['inside_plate']['box'][1] = annotations[id]['inside_plate']['box'][3] - annotations[id]['outside_plate']['height']
    if annotations[id]['inside_plate']['box'][2] >= 0.99999:
        annotations[id]['inside_plate']['box'][2] = annotations[id]['inside_plate']['box'][0] + annotations[id]['outside_plate']['width']
    if annotations[id]['inside_plate']['box'][3] >= 0.99999:
        annotations[id]['inside_plate']['box'][3] = annotations[id]['inside_plate']['box'][1] + annotations[id]['outside_plate']['height']
        
    if annotations[id]['outside_plate']['box'][0] <= 0.00001:
        annotations[id]['outside_plate']['box'][0] = annotations[id]['outside_plate']['box'][2] - annotations[id]['inside_plate']['width']
    if annotations[id]['outside_plate']['box'][1] <= 0.00001:
        annotations[id]['outside_plate']['box'][1] = annotations[id]['outside_plate']['box'][3] - annotations[id]['inside_plate']['height']
    if annotations[id]['outside_plate']['box'][2] >= 0.99999:
        annotations[id]['outside_plate']['box'][2] = annotations[id]['outside_plate']['box'][0] + annotations[id]['inside_plate']['width']
    if annotations[id]['outside_plate']['box'][3] >= 0.99999:
        annotations[id]['outside_plate']['box'][3] = annotations[id]['outside_plate']['box'][1] + annotations[id]['inside_plate']['height']

# save annotations as json
with open('train_data/annotations.json', 'w') as fp:
    json.dump(annotations, fp)


Starting with 995 annotations
Partial width on both sides for 8c823bb58d8cb48e2d6cad6c69ed758d, removing entry...
Partial height on both sides for 86262388700, removing entry...
Partial height on both sides for 14087243500, removing entry...
Missing annotation for 16122796100, removing entry...
Missing annotation for 39876160100, removing entry...
Partial height on both sides for 4265175300, removing entry...
Partial height on both sides for 60006024200, removing entry...
Partial height on both sides for 89483482300, removing entry...
Partial height on both sides for 89496525700, removing entry...
Partial height on both sides for 4258420200, removing entry...
Partial height on both sides for 21314616400, removing entry...
Missing annotation for 28616224900, removing entry...
Partial height on both sides for 61170726800, removing entry...
Partial height on both sides for 100255376100, removing entry...
Partial height on both sides for 37602050500, removing entry...
Partial height on bot